In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.stats as st

from simulation import data_simulation, DICT_SCENARIOS
from models import fit_ci, get_real_data, weighted_quantile, weighted_es

sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2

# Weighted Quantile Coverage Probability

## Real quantile

In [ ]:
SCENARIO = 1  # scenario number
N_REAL = 10000000 # number of real simulations to estimate qW
THETA = 2  # dependence parameter in the Gumbel Copula
ALPHA = 0.5  # risk level between 0 and 1

In [ ]:
X_real, W_real = data_simulation(scenario=SCENARIO, n=N_REAL, theta=THETA, seed=123)

In [ ]:
# Scatter plot of (X,W)
show_data = False
if show_data:
    fig, ax = plt.subplots(figsize=(9,6))
    plt.scatter(X_real, W_real, s=10, edgecolors='black')
    plt.xlabel(r'$X$', fontsize=20)
    plt.ylabel(r'$W$', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    ax.spines["left"].set_color("black")
    ax.spines["bottom"].set_color("black")
    sns.despine()  
    #plt.savefig('imgs/data.jpg')

In [ ]:
# Compute the real quantile
qW_real = weighted_quantile(X_real, W_real, ALPHA)
qW_real

# Confidence interval

In [ ]:
N_REPLICATIONS = 10000  # number of replications
N_SAMPLES = 1000  # number of replication samples
ETA = 0.95  # confidence level

In [ ]:
dict_result = fit_ci(SCENARIO, N_REPLICATIONS, N_SAMPLES, THETA, ALPHA, ETA, qW_real, method='qW')

In [ ]:
coverage_probability = np.mean(dict_result["coverage"])
print(f'The coverage probability of the Weighted Quantile for a risk level {ALPHA*100}% and a confidence level {ETA*100}% is {coverage_probability*100}%')

## Experiments

In [ ]:
# Hyperparameters
# ===============
N_REAL = 10000000  # number of real simulations to estimate qW
THETA = 2  # dependence parameter in the Gumbel Copula
N_REPLICATIONS = 10000  # number of replications
N_SAMPLES = 1000  # number of replication samples
ETA = 0.95  # confidence level
ALPHAS_TEST = [0.05, 0.25, 0.5, 0.75, 0.95]  # risk levels

In [ ]:
# Read qW dictionary
# =================
read_qW_dict = False
if read_qW_dict:
    with open(f"ckpt/qW_n{N_REAL}_theta{THETA}.pickle", 'rb') as fr:
        dict_qW = pickle.load(fr)
    print(dict_qW)

In [ ]:
df_coverage_qW = pd.DataFrame(index = ALPHAS_TEST, columns=DICT_SCENARIOS.keys())
dict_var = {}
for alpha in df_coverage_qW.index:
    for scenario in df_coverage_qW.columns:
        qW_real = get_real_data(scenario=scenario, n=N_REAL, theta=THETA, alpha=alpha, method='qW')
        dict_result = fit_ci(scenario, N_REPLICATIONS, N_SAMPLES, THETA, alpha, ETA, qW_real, method='qW')
        coverage_probability = np.mean(dict_result["coverage"])
        df_coverage_qW.loc[alpha, scenario] = coverage_probability

In [ ]:
df_coverage_qW

In [ ]:
# Plot coverage probability
# =========================
fig, ax = plt.subplots(figsize=(9,6))

# Multiple Scenarios
# -------------------
for scenario in DICT_SCENARIOS.keys():
    plt.scatter(df_coverage_qW.index, df_coverage_qW.loc[:, scenario], s=100, edgecolors='black', label=scenario)

# Confidence level
plt.hlines(ETA, df_coverage_qW.index.min(), df_coverage_qW.index.max(), linestyles='--', color='black')

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
plt.legend(loc='lower left', title='Scenario', fontsize=15, title_fontsize=15)
sns.despine() 
plt.tight_layout()
#plt.savefig(f'imgs/coverage_qW_nreal{N_REAL}_nrep{N_REPLICATIONS}_nsamp{N_SAMPLES}_theta{THETA}_eta{int(ETA*100)}.jpg')

# Weighted Expected Shortfall Coverage Probability

## Real Weighted ES

In [ ]:
# For a fixed alpha, compute the real ES
ALPHA = 0.9
esW_real = weighted_es(X_real, W_real, ALPHA)
esW_real

## Confidence interval

In [ ]:
N_REPLICATIONS = 10000  # number of replications
N_SAMPLES = 1000  # number of replication samples
ETA = 0.95  # confidence level

In [ ]:
dict_result = fit_ci(SCENARIO, N_REPLICATIONS, N_SAMPLES, THETA, ALPHA, ETA, esW_real, method='esW')

In [ ]:
coverage_probability = np.mean(dict_result["coverage"])
print(f'The coverage probability of the Weighted Expected Shortfall for a risk level {ALPHA*100}% and a confidence level {ETA*100}% is {coverage_probability*100}%')

## Experiments

In [ ]:
# Hyperparameters
# ===============
N_REAL = 10000000  # number of real simulations to estimate qW
THETA = 2  # dependence parameter in the Gumbel Copula
N_REPLICATIONS = 10000  # number of replications
N_SAMPLES = 1000  # number of replication samples
ETA = 0.95  # confidence level
ALPHAS_TEST = [0.8, 0.9, 0.95, 0.975]  # risk levels

In [ ]:
# Read esW dictionary
# =================
read_esW_dict = False
if read_esW_dict:
    with open(f"ckpt/esW_n{N_REAL}_theta{THETA}.pickle", 'rb') as fr:
        dict_esW = pickle.load(fr)
    print(dict_esW)

In [ ]:
# Estimate the coverage probability for each scenario
df_coverage_es = pd.DataFrame(index = ALPHAS_TEST, columns=DICT_SCENARIOS.keys())
for alpha in df_coverage_es.index:
    for scenario in df_coverage_es.columns:
        esW_real = get_real_data(scenario=scenario, n=N_REAL, theta=THETA, alpha=alpha, method='esW')
        dict_result = fit_ci(scenario, N_REPLICATIONS, N_SAMPLES, THETA, alpha, ETA, esW_real, method='esW')
        coverage_probability = np.mean(dict_result["coverage"])
        df_coverage_es.loc[alpha, scenario] = coverage_probability

In [ ]:
df_coverage_es

In [ ]:
# Plot coverage probability
# =========================
fig, ax = plt.subplots(figsize=(9,6))

# Multiple Scenarios
# -------------------
for scenario in DICT_SCENARIOS.keys():
    plt.scatter(df_coverage_es.index, df_coverage_es.loc[:, scenario], s=100, edgecolors='black', label=scenario)

# Confidence level
plt.hlines(ETA, df_coverage_es.index.min(), df_coverage_es.index.max(), linestyles='--', color='black')

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
#plt.legend(loc='lower left', title='Scenario', fontsize=15, title_fontsize=15)
sns.despine() 
plt.tight_layout()
#plt.savefig(f'imgs/coverage_esW_nreal{N_REAL}_nrep{N_REPLICATIONS}_nsamp{N_SAMPLES}_theta{THETA}_eta{int(ETA*100)}.jpg')

# Distribution of the errors for 2 risk levels

In [ ]:
ALPHA_1 = 0.5  # risk level of marginal 1
ALPHA_2 = 0.9  # risk level of marginal 2
SCENARIO = 1  # pick a scenario
N_REAL = 10000000  # number of real simulations to estimate qW
N_REPLICATIONS = 10000  # number of replications
N_SAMPLES = 1000  # number of replication samples
# --------------------

# Compute the real quantiles 
qW_real_1 = get_real_data(scenario=SCENARIO, n=N_REAL, theta=THETA, alpha=ALPHA_1)
qW_real_2 = get_real_data(scenario=SCENARIO, n=N_REAL, theta=THETA, alpha=ALPHA_2)

# Fit the estimators
dict_result_1 = fit_ci(scenario, N_REPLICATIONS, N_SAMPLES, THETA, ALPHA_1, ETA, qW_real_1)
qW_hat_1 = dict_result_1['sW_hat']
dict_result_2 = fit_ci(scenario, N_REPLICATIONS, N_SAMPLES, THETA, ALPHA_2, ETA, qW_real_2)
qW_hat_2 = dict_result_2['sW_hat']

In [ ]:
CI_SIDE = 'right'  # confidence interval side: {'left', 'right'}
# ---------------

# Compute the errors
error_1 =  dict_result_1[f'ci_{CI_SIDE}'] - qW_real_1
error_2 =  dict_result_2[f'ci_{CI_SIDE}'] - qW_real_2

# Estimate the parameters of the Gaussian distribution (mean, covariance)
mu_1 = error_1.mean()
mu_2 = error_2.mean()
cov = np.cov(error_1, error_2)

In [ ]:
# Scatter plot
# ===================
h = sns.JointGrid(x=error_1, y=error_2, height=7, ratio=4)
h = h.plot_joint(sns.scatterplot, alpha=0.3, edgecolor='black')

# Make a contour plot
# ===================
xx, yy = np.meshgrid(np.linspace(np.min(error_1)*0.95, np.max(error_1)*1.05, 100),
                     np.linspace(np.min(error_2)*0.95, np.max(error_2)*1.05, 100),)
# 2D-Gaussian distribution with empirical parameters
gaussian_model = st.multivariate_normal(mean=np.stack([mu_1, mu_2]),
                                       cov=cov)
pdfs = gaussian_model.pdf(np.dstack((xx, yy)))
h.ax_joint.contour(xx, yy, pdfs, )
h.refline(x=mu_1, y=mu_2, color='C3')

# Marginal 1
# ==========
h.ax_marg_x.hist(error_1, density=True, bins=50, edgecolor='black')
h.ax_marg_x.plot(np.sort(error_1), st.norm(mu_1, np.sqrt(cov[0][0])).pdf(np.sort(error_1)), color='C1', linewidth=2)

# Marginal 2
# ==========
h.ax_marg_y.hist(error_2, density=True, bins=50, orientation="horizontal", edgecolor='black')
h.ax_marg_y.plot(st.norm(mu_2, np.sqrt(cov[1][1])).pdf(np.sort(error_2)), np.sort(error_2), color='C1', linewidth=2,)

h.ax_joint.set_label(None)
h.ax_joint.spines["left"].set_color("black")
h.ax_joint.spines["bottom"].set_color("black")
h.ax_joint.set_xlabel(None)
h.ax_joint.set_ylabel(None)
h.figure.tight_layout()

# plt.savefig(f'imgs/dist_erros_a1-{int(ALPHA_1*100)}_a1-{int(ALPHA_2*100)}_scenario-{SCENARIO}_side-{CI_SIDE}.jpg')